# Exercices

## **Préliminaires**: Clone de votre repo et imports

In [7]:
! git clone https://github.com/Zakaria-Yahya/exam_2025.git
! cp exam_2025/utils/utils_exercices.py .

import copy
import numpy as np
import torch

fatal: destination path 'exam_2025' already exists and is not an empty directory.


**Clef personnelle pour la partie théorique**

Dans la cellule suivante, choisir un entier entre 100 et 1000 (il doit être personnel). Cet entier servira de graine au générateur de nombres aléatoire a conserver pour tous les exercices.



In [8]:
mySeed = 388

\

---

\

\

**Exercice 1** *Une relation linéaire*

La fonction *generate_dataset* fournit deux jeux de données (entraînement et test). Pour chaque jeu de données, la clef 'inputs' donne accès à un tableau numpy (numpy array) de prédicteurs empilés horizontalement : chaque ligne $i$ contient trois prédicteurs $x_i$, $y_i$ et $z_i$. La clef 'targets' renvoie le vecteur des cibles $t_i$. \

Les cibles sont liées aux prédicteurs par le modèle:
$$ t = \theta_0 + \theta_1 x + \theta_2 y + \theta_3 z + \epsilon$$ où $\epsilon \sim \mathcal{N}(0,\eta)$


In [33]:
from utils_exercices import generate_dataset, Dataset1
train_set, test_set = generate_dataset(mySeed)

**Q1** Par quelle méthode simple peut-on estimer les coefficients $\theta_k$ ? La mettre en oeuvre avec la librairie python de votre choix.

Réponse:
La méthode la plus simple et la plus efficace pour estimer les coefficients (θ) dans un modèle de régression linéaire comme celui-ci est d'utiliser les moindres carrés ordinaires (OLS).

In [34]:
# Extraire les entrées (X) et les cibles (y) de l'ensemble d'apprentissage
X = torch.tensor(train_set['inputs'], dtype=torch.float32)
y = torch.tensor(train_set['targets'], dtype=torch.float32)

# Ajouter une colonne de uns à X pour le terme d'interception (θ0)
X = torch.cat([torch.ones(X.shape[0], 1), X], dim=1)

# Calculer les coefficients OLS à l'aide de la formule : θ = (X^T X)^-1 X^T y
theta = torch.linalg.solve(X.T @ X, X.T @ y)

# Afficher les coefficients estimés
print("Coefficients estimés (θ) :", theta)

# Calculate predictions
predictions = X @ theta

# Calculate error (Mean Squared Error - MSE)
error = torch.mean((predictions - y)**2)

# Print the error
print("Mean Squared Error (MSE):", error)

Coefficients estimés (θ) : tensor([19.3857,  3.9040,  3.8649,  7.5905])
Mean Squared Error (MSE): tensor(3.9825)


**Q2** Dans les cellules suivantes, on se propose d'estimer les $\theta_k$ grâce à un réseau de neurones entraîné par SGD. Quelle architecture s'y prête ? Justifier en termes d'expressivité et de performances en généralisation puis la coder dans la cellule suivante.

Réponse:

Expressivité : Une seule couche linéaire est suffisante pour apprendre une relation linéaire. Ajouter des couches supplémentaires ou des non-linéarités serait superflu et risquerait de dégrader les performances en généralisation.

Performances en généralisation : Une architecture simple réduit le risque de sur-apprentissage (overfitting), surtout avec un bruit gaussien (ϵ∼N(0,η)). Cela garantit une bonne performance sur des données non vues.

In [35]:
# Dataset et dataloader :
dataset = Dataset1(train_set['inputs'], train_set['targets'])
dataloader = torch.utils.data.DataLoader(dataset, batch_size=100, shuffle=True)
import torch.nn as nn

# A coder :
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc = nn.Linear(3, 1)

    def forward(self, x):
        return self.fc(x)

**Q3** Entraîner cette architecture à la tâche de régression définie par les entrées et sorties du jeu d'entraînement (compléter la cellule ci-dessous).

In [36]:

# Initialize model, loss, and optimizer
mySimpleNet = SimpleNet()
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(mySimpleNet.parameters(), lr=0.01)

# Training loop
num_epochs = 500
for epoch in range(num_epochs):
    for batch_inputs, batch_targets in dataloader:

        # Mise en mode entraînement
        mySimpleNet.train()

        # Étape forward : calcul des prédictions
        predictions = mySimpleNet(batch_inputs)

        # Calcul de la perte
        loss = criterion(predictions, batch_targets.unsqueeze(1))  # Ajouter une dimension pour le batch

        # Étape backward : calcul des gradients
        optimizer.zero_grad()  # Remise à zéro des gradients
        loss.backward()  # Calcul des gradients

        # Mise à jour des paramètres
        optimizer.step()

    # Affichage de la perte pour suivre l'entraînement
    if (epoch + 1) % 50 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')


Epoch [50/500], Loss: 3.7808
Epoch [100/500], Loss: 5.1837
Epoch [150/500], Loss: 3.2892
Epoch [200/500], Loss: 3.6998
Epoch [250/500], Loss: 3.9494
Epoch [300/500], Loss: 5.0093
Epoch [350/500], Loss: 4.0476
Epoch [400/500], Loss: 3.1325
Epoch [450/500], Loss: 4.0927
Epoch [500/500], Loss: 3.4747


**Q4** Où sont alors stockées les estimations des  $\theta_k$ ? Les extraire du réseau *mySimpleNet* dans la cellule suivante.

In [37]:
# Accéder aux paramètres de la couche linéaire
linear_layer_params = mySimpleNet.fc.parameters()

# Extraire les poids et le biais
weights = next(iter(linear_layer_params)).detach().numpy()  # poids (θ1, θ2, θ3)
bias = next(iter(linear_layer_params)).detach().numpy()  # biais (θ0)

# Afficher les estimations des coefficients
print("Poids (θ1, θ2, θ3) :", weights)
print("Biais (θ0) :", bias)

Poids (θ1, θ2, θ3) : [[3.904183  3.8635218 7.58983  ]]
Biais (θ0) : [19.384892]


**Q5** Tester ces estimations sur le jeu de test et comparer avec celles de la question 1. Commentez.

In [38]:

# Charger les données de test
test_inputs = test_set['inputs']  # Prédicteurs du jeu de test
test_targets = test_set['targets']  # Cibles réelles du jeu de test

# Déplacer les données sur le même appareil que le modèle
test_inputs = torch.tensor(test_inputs, dtype=torch.float32)

# Générer les prédictions à l'aide du modèle
mySimpleNet.eval()  # Mettre le modèle en mode évaluation
with torch.no_grad():  # Pas de calcul de gradients
    predictions = mySimpleNet(test_inputs).numpy().flatten()  # Prédictions aplaties

# Calculer l'erreur quadratique moyenne (MSE) entre les prédictions et les cibles
mse_test = np.mean((predictions - test_targets) ** 2)

# Affichage des résultats
print(f"Erreur quadratique moyenne (MSE) sur le jeu de test : {mse_test:.4f}")

# Comparaison avec les résultats obtenus dans la question 1
# Si vous avez les prédictions ou coefficients précédents, insérez-les ici pour comparaison


Erreur quadratique moyenne (MSE) sur le jeu de test : 3.8208


Comparaison des erreurs quadratiques moyennes (MSE) :

MSE sur le jeu de test avec le réseau de neurones : 3.8208\
MSE obtenu dans la question 1  :
3.9825

La MSE obtenue avec le réseau de neurones est légèrement inférieure à celle obtenue dans la question 1, ce qui montre que la deuxième méthode est plus efficace mais la première est plus optimale en terme de temps.

**Exercice 2** *Champ réceptif et prédiction causale*

Le réseau défini dans la cellule suivante est utilisé pour faire le lien entre les valeurs $(x_{t' \leq t})$ d'une série temporelle d'entrée et la valeur présente $y_t$ d'une série temporelle cible.

In [39]:
import torch.nn as nn
import torch.nn.functional as F
from utils_exercices import Outconv, Up_causal, Down_causal

class Double_conv_causal(nn.Module):
    '''(conv => BN => ReLU) * 2, with causal convolutions that preserve input size'''
    def __init__(self, in_ch, out_ch, kernel_size=3, dilation=1):
        super(Double_conv_causal, self).__init__()
        self.kernel_size = kernel_size
        self.dilation = dilation
        self.conv1 = nn.Conv1d(in_ch, out_ch, kernel_size=kernel_size, padding=0, dilation=dilation)
        self.bn1 = nn.BatchNorm1d(out_ch)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv1d(out_ch, out_ch, kernel_size=kernel_size, padding=0, dilation=dilation)
        self.bn2 = nn.BatchNorm1d(out_ch)

    def forward(self, x):
        x = F.pad(x, ((self.kernel_size - 1) * self.dilation, 0))
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)

        x = F.pad(x, ((self.kernel_size - 1) * self.dilation, 0))
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        return x


class causalFCN(nn.Module):
    def __init__(self, dilation=1):
        super(causalFCN, self).__init__()
        size = 64
        n_channels = 1
        n_classes = 1
        self.inc = Double_conv_causal(n_channels, size)
        self.down1 = Down_causal(size, 2*size)
        self.down2 = Down_causal(2*size, 4*size)
        self.down3 = Down_causal(4*size, 8*size, pooling_kernel_size=5, pooling_stride=5)
        self.down4 = Down_causal(8*size, 4*size, pooling=False, dilation=2)
        self.up2 = Up_causal(4*size, 2*size, kernel_size=5, stride=5)
        self.up3 = Up_causal(2*size, size)
        self.up4 = Up_causal(size, size)
        self.outc = Outconv(size, n_classes)
        self.n_classes = n_classes

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up2(x5, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        x = self.outc(x)
        return x

# Exemple d'utilisation
model = causalFCN()
# Série temporelle d'entrée (x_t):
input_tensor1 = torch.rand(1, 1, 10000)
# Série temporelle en sortie f(x_t):
output = model(input_tensor1)
print(output.shape)

torch.Size([1, 1, 10000])


**Q1** De quel type de réseau de neurones s'agit-il ? Combien de paramètres la couche self.Down1 compte-t-elle (à faire à la main) ?
Combien de paramètres le réseau entier compte-t-il (avec un peu de code) ?

In [41]:
#Type de réseau : Le réseau présenté est un Fully Convolutional Network (FCN).
# Nb de paramètres dans self.Down1: (calcul "à la main")
# Nombre de paramètres = 64×128×3 + 128 = 24704
# Nb de paramètres au total:
# Calculer le nombre total de paramètres dans le réseau
total_params = sum(p.numel() for p in model.parameters())
print(f"Nombre total de paramètres du réseau : {total_params}")

Nombre total de paramètres du réseau : 2872641


**Q2** Par quels mécanismes la taille du vecteur d'entrée est-elle réduite ? Comment est-elle restituée dans la deuxième partie du réseau ?

Reponse:

La taille du vecteur d'entrée est réduite à traver la fonction Down_causal.\
Elle restutié à l'aide de Up_causal

**Q3** Par quels mécanismes le champ réceptif est-il augmenté ? Préciser par un calcul la taille du champ réceptif en sortie de *self.inc*.

Le champ réceptif est augmenté par l'empilement de couches de convolution et la dilatation, permettant aux neurones de capter des informations sur une zone plus large de l'entrée. La couche self.inc, composée de deux convolutions causales avec un kernel_size de 3 et une dilation de 1, a un champ réceptif en sortie de 5, calculé comme suit: 1 (initial) + 3 (1ère convolution) + (3-1) (2ème convolution) = 5.

**Q4** Par un bout de code, déterminer empiriquement la taille du champ réceptif associé à la composante $y_{5000}$ du vecteur de sortie. (Indice: considérer les sorties associées à deux inputs qui ne diffèrent que par une composante...)

In [42]:
import torch

# Créer deux entrées qui ne diffèrent que par une composante
input1 = torch.zeros(1, 1, 10000)
input2 = input1.clone()
input2[0, 0, 5000] = 1  # Modifier une seule composante

# Obtenir les sorties du modèle pour les deux entrées
model.eval()  # Mettre le modèle en mode évaluation
with torch.no_grad():
    output1 = model(input1)
    output2 = model(input2)

# Calculer la différence entre les sorties
diff = output2 - output1

# Trouver les indices des composantes non nulles dans la différence
receptive_field_indices = torch.nonzero(diff.squeeze())

# Calculer la taille du champ réceptif
receptive_field_size = receptive_field_indices[-1] - receptive_field_indices[0] + 1

# Afficher la taille du champ réceptif
print("Taille du champ réceptif :", receptive_field_size.item())

Taille du champ réceptif : 305


**Q5** $y_{5000}$ dépend-elle des composantes $x_{t, \space t > 5000}$ ? Justifier de manière empirique puis préciser la partie du code de Double_conv_causal qui garantit cette propriété de "causalité" en justifiant.  



Non, $𝑦_{5000}$ ne dépend pas des composantes $x_{t, \space t > 5000}$. Le champ réceptif empirique est de 305, donc $𝑦_{5000}$ dépend uniquement de 𝑥𝑡 pour 4696≤𝑡≤5000 . Le padding causal dans Double_conv_causal, garantit la causalité en ajoutant des zéros uniquement du côté gauche de l'entrée, empêchant la convolution de "voir" les valeurs futures.

\

---

\

\

Exercice 3: "Ranknet loss"

Un [article récent](https://https://arxiv.org/abs/2403.14144) revient sur les progrès en matière de learning to rank. En voilà un extrait :

texte en italique
<img src="https://raw.githubusercontent.com/nanopiero/exam_2025/refs/heads/main/utils/png_exercice3.PNG?token=GHSAT0AAAAAAC427DACOPGNDNN6UDOLVLLAZ4BB2JQ" alt="extrait d'un article" width="800">

**Q1** Qu'est-ce que les auteurs appellent "positive samples" et "negative samples" ? Donner un exemple.

Réponse:

Les "positive samples" désignent les échantillons qui sont jugés pertinents pour une requête donnée, tandis que les "negative samples" désignent ceux qui sont jugés non pertinents.

**Q2** Dans l'expression de $\mathcal{L}_{RankNet}$, d'où proviennent les $z_i$ ? Que représentent-ils ?  

Réponse:
Les zi
  proviennent du modèle de prédiction. Ils représentent les scores prévus par le modèle pour chaque échantillon.




**Q3** Pourquoi cette expression conduit-elle à ce que, après apprentissage, "the estimated
value of positive samples is greater than that of negative samples
for each pair of positive/negative samples" ?

L'expression $\mathcal{L}_{RankNet}$ inclut une fonction log-sigmoïde (
𝜎(𝑧𝑖 − 𝑧𝑗)
 )) qui maximise la probabilité que le score
zi  soit supérieur à zj
  pour un échantillon positif (yij = 1) et minimise cette probabilité pour un échantillon négatif (yij = 0). Par conséquent, l'optimisation de cette fonction de perte pousse le modèle à produire des scores (zi > zj)
  pour des paires positives/négatives, ce qui garantit que les échantillons positifs obtiennent des scores supérieurs aux échantillons négatifs.

**Q4** Dans le cadre d'une approche par deep learning, quels termes utilise-t-on pour qualifier les réseaux de neurones exploités et la modalité suivant laquelle ils sont entraînés ?